In [3]:
cd(@__DIR__) #makes the directory where this script is located the new working directory
using Pkg
Pkg.activate()
Pkg.instantiate()
using MixedModels
using RCall
using DataFrames
using CSV
using RData
using Statistics
using Dates

R"""
require(dplyr, quietly = TRUE)   # for data wrangling
require(tidyverse, quietly = TRUE)   # for data wrangling
require(lme4)
require(lmerTest)
require(quickpsy)

SimulatePsychometricFunction_Staircase = function(ID, 
                                                  ConditionOfInterest, 
                                                  StandardValues, 
                                                  reps, 
                                                  PSE_Difference, 
                                                  JND_Difference, 
                                                  Multiplicator_PSE_Standard, 
                                                  Multiplicator_SD_Standard, 
                                                  SD_ResponseFunction, 
                                                  Mean_Variability_Between = 0.1, 
                                                  SD_Variability_Between = 0.1){
  Psychometric = expand.grid(ID=ID, ConditionOfInterest=ConditionOfInterest, StandardValues=StandardValues, reps = reps)
  
  Psychometric = Psychometric %>%
    group_by(ID) %>%#
    mutate(PSE_Factor_ID = rnorm(1,1,Mean_Variability_Between),
           SD_Factor_ID = rnorm(1,1,SD_Variability_Between))
  
  Psychometric = Psychometric %>%
    mutate(
      Mean_Standard = StandardValues+StandardValues*Multiplicator_PSE_Standard,
      SD_Standard = StandardValues*Multiplicator_SD_Standard,
      Mean = (Mean_Standard + (ConditionOfInterest==ConditionOfInterest[2])*StandardValues*PSE_Difference)*PSE_Factor_ID,
      SD = abs((SD_Standard + (ConditionOfInterest==ConditionOfInterest[2])*SD_Standard*JND_Difference)*SD_Factor_ID),
      staircase_factor = rcauchy(length(reps),1,SD_ResponseFunction), 
      Presented_TestStimulusStrength = Mean*staircase_factor,
      Difference = Presented_TestStimulusStrength - StandardValues,
      AnswerProbability = pnorm(Presented_TestStimulusStrength,Mean,SD),
      Answer = as.numeric(rbernoulli(length(AnswerProbability),AnswerProbability))
    )
  
  Psychometric = Psychometric %>%
    filter(abs(staircase_factor-1) < 0.75) %>%
    group_by(ID,ConditionOfInterest,StandardValues,Difference) %>%
    mutate(Yes = sum(Answer==1),
           Total = length(ConditionOfInterest))
  
  Psychometric
}
""";

 Activating environment at `C:\Users\bjoer\.julia\environments\v1.4\Project.toml`


R version 3.6.1 (2019-07-05) -- "Action of the Toes"
Copyright (C) 2019 The R Foundation for Statistical Computing
Platform: x86_64-w64-mingw32/x64 (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.



Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.



┌ Warning: RCall.jl: [Previously saved workspace restored]
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning: package 'dplyr' was built under R version 3.6.3
│ 
│ Attaching package: 'dplyr'
│ 
│ The following objects are masked from 'package:stats':
│ 
│     filter, lag
│ 
│ The following objects are masked from 'package:base':
│ 
│     intersect, setdiff, setequal, union
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning: package 'tidyverse' was built under R version 3.6.3
│ -- Attaching packages --------------------------------------- tidyverse 1.3.0 --
│ v ggplot2 3.3.0     v purrr   0.3.3
│ v tibble  3.0.0     v stringr 1.4.0
│ v tidyr   1.0.2     v forcats 0.5.0
│ v readr   1.3.1     
│ Warning: package 'ggplot2' was built under R version 3.6.3
│ Warning: package 'tibble' was built under R version 3.6.3
│ Warning: package 'tidyr' was built under R version 3.6.3
│ Warning: package 'forc

In [5]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
Range_reps = [30,40,50,60]
PSE_Difference = 0.1
JND_Difference = 0.2
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 25
Range_Participants = [10,12,14,16,18,20]

TotalNumber = length(Range_reps)*length(Range_Participants)
CurrentRunthrough = 0
rightnow = Dates.now()
formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference + ConditionOfInterest |ID) + (Difference + ConditionOfInterest|StandardValues));

for reps in Range_reps
    for n in Range_Participants
        CurrentRunthrough = CurrentRunthrough + 1

        if CurrentRunthrough > 1
        print("This runthrough took ", Dates.now()-TimeStartRunthrough, " s")                
        end
        
        print("Starting runthrough N° ", CurrentRunthrough, " out of ", TotalNumber)
        global TimeStartRunthrough = Dates.now()

        for j in 1:nIterations
            
            @rput reps n
            
            R"""
            ID = paste0("s",1:n)
            
            Psychometric = SimulatePsychometricFunction_Staircase(ID,
                            ConditionOfInterest,
                            StandardValues,
                            1:reps,
                            PSE_Difference,
                            JND_Difference,
                            Multiplicator_PSE_Standard,
                            Multiplicator_SD_Standard,
                            SD_ResponseFunction,
                            Mean_Variability_Between,
                            SD_Variability_Between)

            Psychometric$StandardValues = as.character(Psychometric$StandardValues)

            """
                    
            @rget Psychometric
            
            TimeStartTrial = Dates.now()
            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
            GLMM_Model.optsum.optimizer = :LN_NELDERMEAD
            GLMM = fit!(GLMM_Model, fast=true)
            DurationGLMM_NeldMeader_AGP0 = ((Dates.now()) - TimeStartTrial)
            AIC_NeldMeader_AGP0 = aic(GLMM)
            Pvalues_NeldMeader_AGP0 = [(coeftable(GLMM)).cols[4][2];(coeftable(GLMM)).cols[4][4]]   
#            
#            TimeStartTrial = Dates.now()
#            formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference + ConditionOfInterest |ID) + (Difference + ConditionOfInterest|StandardValues));
#            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
#            GLMM_Model.optsum.optimizer = :LN_NELDERMEAD
#            GLMM = fit!(GLMM_Model, fast=false)
#            DurationGLMM_NeldMeader_AGP1 = ((Dates.now()) - TimeStartTrial)
#            AIC_NeldMeader_AGP1 = aic(GLMM)
#            Pvalues_NeldMeader_AGP1 = [(coeftable(GLMM)).cols[4][2];(coeftable(GLMM)).cols[4][4]]

            TimeStartTrial = Dates.now()
            GLMM = fit!(GeneralizedLinearMixedModel(formula1, Psychometric, Binomial()), fast=true) #default optimizer is bobyqa
            DurationGLMM_bobyqa_AGP0 = ((Dates.now()) - TimeStartTrial)
            AIC_bobyqa_AGP0 = aic(GLMM)
            Pvalues_bobyqa_AGP0 = [(coeftable(GLMM)).cols[4][2];(coeftable(GLMM)).cols[4][4]]
                    
#            TimeStartTrial = Dates.now()
#            formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference + ConditionOfInterest |ID) + (Difference + ConditionOfInterest|StandardValues));
#            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
#            #default optimizer is bobyqa
#            GLMM = fit!(GLMM_Model, fast=false)
#            DurationGLMM_bobyqa_AGP1 = ((Dates.now()) - TimeStartTrial)
#            AIC_bobyqa_AGP1 = aic(GLMM)
#            Pvalues_bobyqa_AGP1 = [(coeftable(GLMM)).cols[4][2];(coeftable(GLMM)).cols[4][4]]

            if CurrentRunthrough == 1 && j == 1 && reps == Range_reps[1] && n == Range_Participants[1]
                global PowerfulDataframe = DataFrame(
                        nIteration = j,
                        n = n,
                        reps = reps,
                        
                        DurationGLMM_NeldMeader_AGP0=DurationGLMM_NeldMeader_AGP0, 
                        AIC_NeldMeader_AGP0=AIC_NeldMeader_AGP0, 
                        Pvalues_NeldMeader_AGP0_Accuracy=Pvalues_NeldMeader_AGP0[1],
                        Pvalues_NeldMeader_AGP0_Precision=Pvalues_NeldMeader_AGP0[2],
                    
#                        DurationGLMM_NeldMeader_AGP1=DurationGLMM_NeldMeader_AGP1,
#                        AIC_NeldMeader_AGP1=AIC_NeldMeader_AGP1,
#                        Pvalues_NeldMeader_AGP1=Pvalues_NeldMeader_AGP1,
                    
                        DurationGLMM_bobyqa_AGP0=DurationGLMM_bobyqa_AGP0, 
                        AIC_bobyqa_AGP0=AIC_bobyqa_AGP0, 
                        Pvalues_bobyqa_AGP0_Accuracy=Pvalues_bobyqa_AGP0[1],
                        Pvalues_bobyqa_AGP0_Precision=Pvalues_bobyqa_AGP0[2],
                    
#                        DurationGLMM_bobyqa_AGP1=DurationGLMM_bobyqa_AGP1, 
#                        AIC_bobyqa_AGP1=AIC_bobyqa_AGP1, 
#                        Pvalues_bobyqa_AGP1 = Pvalues_bobyqa_AGP1,
                
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],StandardValue2=StandardValues[2], 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard,
                        SD_ResponseFunction = SD_ResponseFunction,
                        Mean_Variability_Between = Mean_Variability_Between,
                        SD_Variability_Between = SD_Variability_Between)

            else
                row = DataFrame(
                        nIteration = j,
                        n = n,
                        reps = reps,
                        
                        DurationGLMM_NeldMeader_AGP0=DurationGLMM_NeldMeader_AGP0, 
                        AIC_NeldMeader_AGP0=AIC_NeldMeader_AGP0,
                        Pvalues_NeldMeader_AGP0_Accuracy=Pvalues_NeldMeader_AGP0[1],
                        Pvalues_NeldMeader_AGP0_Precision=Pvalues_NeldMeader_AGP0[2],
                    
#                        DurationGLMM_NeldMeader_AGP1=DurationGLMM_NeldMeader_AGP1,
#                        AIC_NeldMeader_AGP1=AIC_NeldMeader_AGP1,
#                        Pvalues_NeldMeader_AGP1=Pvalues_NeldMeader_AGP1,
                    
                        DurationGLMM_bobyqa_AGP0=DurationGLMM_bobyqa_AGP0, 
                        AIC_bobyqa_AGP0=AIC_bobyqa_AGP0, 
                        Pvalues_bobyqa_AGP0_Accuracy=Pvalues_bobyqa_AGP0[1],
                        Pvalues_bobyqa_AGP0_Precision=Pvalues_bobyqa_AGP0[2],
                    
#                        DurationGLMM_bobyqa_AGP1=DurationGLMM_bobyqa_AGP1, 
#                        AIC_bobyqa_AGP1=AIC_bobyqa_AGP1, 
#                        Pvalues_bobyqa_AGP1 = Pvalues_bobyqa_AGP1,
                
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],
                        StandardValue2=StandardValues[2], 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard,
                        SD_ResponseFunction = SD_ResponseFunction,
                        Mean_Variability_Between = Mean_Variability_Between,
                        SD_Variability_Between = SD_Variability_Between)
                
                PowerfulDataframe = append!(PowerfulDataframe,row)
            end
        end
    end
end

CSV.write(join(["Durations_Julia.csv"]),PowerfulDataframe)

Starting runthrough N° 1 out of 24This runthrough took 909 milliseconds sStarting runthrough N° 2 out of 24This runthrough took 1061 milliseconds sStarting runthrough N° 3 out of 24This runthrough took 678 milliseconds sStarting runthrough N° 4 out of 24This runthrough took 1031 milliseconds sStarting runthrough N° 5 out of 24This runthrough took 873 milliseconds sStarting runthrough N° 6 out of 24This runthrough took 1375 milliseconds sStarting runthrough N° 7 out of 24This runthrough took 604 milliseconds sStarting runthrough N° 8 out of 24This runthrough took 626 milliseconds sStarting runthrough N° 9 out of 24This runthrough took 896 milliseconds sStarting runthrough N° 10 out of 24This runthrough took 3949 milliseconds sStarting runthrough N° 11 out of 24This runthrough took 1958 milliseconds sStarting runthrough N° 12 out of 24This runthrough took 2187 milliseconds sStarting runthrough N° 13 out of 24This runthrough took 879 milliseconds sStarting runthrough N° 14 out of 24This r

In [4]:
PowerfulDataframe

,nIteration,n,reps,DurationGLMM_NeldMeader_AGP0,AIC_NeldMeader_AGP0,Pvalues_NeldMeader_AGP0_Accuracy
,Int64,Int64,Int64,Millisec…,Float64,Float64
1,1,10,30,123211 milliseconds,4077.32,4.30388e-14
2,1,12,30,56875 milliseconds,4036.01,9.7691e-10
3,1,14,30,37153 milliseconds,4169.71,4.58931e-9
4,1,16,30,48692 milliseconds,4118.0,1.40641e-9
5,1,18,30,42031 milliseconds,3976.38,2.93625e-14
6,1,20,30,29418 milliseconds,4168.15,4.36978e-11
7,1,10,40,53537 milliseconds,4075.99,1.1792e-14
8,1,12,40,36506 milliseconds,4012.85,1.19685e-14
9,1,14,40,42479 milliseconds,4033.97,2.51751e-9


123

ArgumentError: ArgumentError: invalid NLopt arguments: zero step size